In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Unlimited columns

In [3]:
from scripts import (
    load_uci, load_tidepool_dummy, load_so_pump, load_so_cgm
)

In [4]:
df = load_so_pump()
print(df.shape)
df.head()

(4507, 30)


,C0,C1,C2,COMMENT,D0,D1,D2,D3,D4,DATEEVENT,DEVICE_ID,DEVICE_MODEL,EVENTTYPE,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,ISMANUAL,KEY0,KEY1,KEY2,TIMESLOT,VENDOR_EVENT_ID,VENDOR_EVENT_TYPE_ID
0,NaN,NaN,NaN,NaN,0,0,0,0,0,43331.6085069444,130449095,OmniPod PDM,1,0,100,0,0,0,0,0,0,0,0,0,100,0,0,3,0-H5,0
1,NaN,NaN,NaN,NaN,0,0,0,0,0,43333.4511574074,130449095,OmniPod PDM,1,0,88,0,0,0,0,0,0,0,0,0,88,0,0,2,0-H36,0
2,NaN,NaN,NaN,NaN,0,0,0,0,0,43334.5869791667,130449095,OmniPod PDM,1,0,162,0,0,0,0,0,0,0,0,0,162,0,0,3,0-H58,0
3,NaN,NaN,NaN,NaN,0,0,0,0,0,43336.8517824074,130449095,OmniPod PDM,1,0,95,0,0,0,0,0,0,0,0,0,95,0,0,5,0-H85,0
4,NaN,NaN,NaN,NaN,0,0,0,0,0,43337.5129513889,130449095,OmniPod PDM,1,0,161,0,0,0,0,0,0,0,0,0,161,0,0,3,0-H96,0


In [5]:
df = load_so_cgm()
print(df.shape)
df.head()

(92453, 14)


,Index,Timestamp (YYYY-MM-DDThh:mm:ss),Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer),Transmitter ID
9,10,2018-07-22T00:04:51,EGV,NaN,NaN,NaN,iPhone,168,NaN,NaN,NaN,NaN,8413883.0,4G23WB
10,11,2018-07-22T00:09:51,EGV,NaN,NaN,NaN,iPhone,154,NaN,NaN,NaN,NaN,8414183.0,4G23WB
11,12,2018-07-22T00:14:51,EGV,NaN,NaN,NaN,iPhone,144,NaN,NaN,NaN,NaN,8414483.0,4G23WB
12,13,2018-07-22T00:19:51,EGV,NaN,NaN,NaN,iPhone,137,NaN,NaN,NaN,NaN,8414783.0,4G23WB
13,14,2018-07-22T00:24:51,EGV,NaN,NaN,NaN,iPhone,130,NaN,NaN,NaN,NaN,8415083.0,4G23WB


## Data Cleaning

In [ ]:
X_train, X_val, y_train, y_val = \
    train_test_split(df_train.drop(['Fail'], axis=1), 
                     df_train['Fail'], 
                     test_size=0.2)
X_train.columns.values

In [ ]:
param_grid = {
    'learning_rate': [0.06],
    'n_estimators':  [80],
    'max_depth': [3],
    'subsample': [0.9],
    'colsample_bytree': [0.3],
    'gamma': [4, 5]
}

gridsearch = GridSearchCV(XGBClassifier(),
                          param_grid=param_grid, 
                          scoring='roc_auc', cv=3, n_jobs=-1,
                          return_train_score=True, verbose=10)

gridsearch.fit(X_train, y_train)